## trabajo con datasets nuevos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from pandas import PeriodIndex

## tener en cuenta que la tabla principal (motor-vehicle-collisions-crashes.csv) es la union (supuestamente) de la tabla de colision entre vehiculos y colision con persona

## motor-vehicle-collisions-crashes.csv

https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95

In [ ]:
df_vehicle_crash = pd.read_csv(r"content\drive\Mi Unidad\PG\Semana 2\clean-datasets\motor-vehicle-collisions-crashes.csv")

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_vehicle_crash.sort_values(["CRASH_DATE", "CRASH_TIME"], inplace = True)

df_vehicle_crash.reset_index(drop = True, inplace = True)

df_vehicle_crash

# tengo muchos NaN, no puedo hacer .dropna() porque vacio todo el dataset sino
# deberia hacer un .fillna(<string>) pero no se con que rellenar

In [ ]:
df_vehicle_crash.columns

PREGUNTAS DE KPI:

    porcentaje de choques por día de la semana (por mes)
    porcentaje de choques por hora (por día de la semana y mes)
    cual es la zona con mas accidentes (por mes, dia y hora)

FEATURES IMPORTANTES:

    CRASH_DATE, CRASH_TIME, BOROUGH, LATITUDE, LONGITUDE, ON_STREET_NAME, CROSS_STREET_NAME, OFF_STREET_NAME

FEATURES DE INFORMACION:

    NUMBER_OF_PERSONS_INJURED, NUMBER_OF_PERSONS_KILLED, NUMBER_OF_PEDESTRIANS_INJURED, NUMBER_OF_PEDESTRIANS_KILLED, NUMBER_OF_CYCLIST_INJURED, NUMBER_OF_CYCLIST_KILLED, NUMBER_OF_MOTORIST_INJURED, NUMBER_OF_MOTORIST_KILLED, 'CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2', 'CONTRIBUTING_FACTOR_VEHICLE_3', 'CONTRIBUTING_FACTOR_VEHICLE_4', 'CONTRIBUTING_FACTOR_VEHICLE_5'

AGREGAR:

    WEEK_DAY

RELACION CON OTRAS TABLAS:

    'COLLISION_ID'

In [ ]:
# me quedo con las features que considero importantes
df_vehicle_crash_clean = df_vehicle_crash[[ 'CRASH_DATE', 'CRASH_TIME', 'COLLISION_ID', 'BOROUGH', 'ZIP_CODE', 'LATITUDE', 
                                            'LONGITUDE', 'LOCATION', 'ON_STREET_NAME', 'CROSS_STREET_NAME', 
                                            'OFF_STREET_NAME', 'NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED', 
                                            'NUMBER_OF_PEDESTRIANS_INJURED', 'NUMBER_OF_PEDESTRIANS_KILLED', 
                                            'NUMBER_OF_CYCLIST_INJURED', 'NUMBER_OF_CYCLIST_KILLED', 'NUMBER_OF_MOTORIST_INJURED',
                                            'NUMBER_OF_MOTORIST_KILLED', 'CONTRIBUTING_FACTOR_VEHICLE_1',
                                            'CONTRIBUTING_FACTOR_VEHICLE_2', 'CONTRIBUTING_FACTOR_VEHICLE_3',
                                            'CONTRIBUTING_FACTOR_VEHICLE_4', 'CONTRIBUTING_FACTOR_VEHICLE_5']]

df_vehicle_crash_clean.sort_values("COLLISION_ID").head()

In [ ]:
# hago la captura del dia de la semana

dates = df_vehicle_crash_clean["CRASH_DATE"]

idx = pd.PeriodIndex(dates, freq = "D") #configuro la frecuencia para que sea por dia

week_day = idx.dayofweek #asigno la configuracion de numero de dia

# week_day

df_vehicle_crash_clean["WEEK_DAY"] = week_day

# df_vehicle_crash_clean

#lunes = 0
#martes = 1
#miercoles = 2
#jueves = 3
#viernes = 4

acomodo el dia de la semana para que quede entre las primeras columnas

In [ ]:
df_vehicle_crash_clean = df_vehicle_crash_clean[[   "CRASH_DATE", "CRASH_TIME", "WEEK_DAY", "BOROUGH", "LATITUDE", 
                                                    "LONGITUDE", "ON_STREET_NAME", "CROSS_STREET_NAME", "OFF_STREET_NAME",
                                                    "NUMBER_OF_PERSONS_INJURED", "NUMBER_OF_PERSONS_KILLED", 
                                                    "NUMBER_OF_PEDESTRIANS_INJURED", "NUMBER_OF_PEDESTRIANS_KILLED", 
                                                    "NUMBER_OF_CYCLIST_INJURED", "NUMBER_OF_CYCLIST_KILLED", 
                                                    "NUMBER_OF_MOTORIST_INJURED", "NUMBER_OF_MOTORIST_KILLED", 
                                                    'CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2', 
                                                    'CONTRIBUTING_FACTOR_VEHICLE_3', 'CONTRIBUTING_FACTOR_VEHICLE_4', 
                                                    'CONTRIBUTING_FACTOR_VEHICLE_5'  ]]

df_vehicle_crash_clean.head()

muestro la cantidad de registros faltantes y que porcentaje del total representan

In [ ]:
# celda del codigo de jeampierr muy interesante

mask_crash = df_vehicle_crash_clean.isnull()  #obtengo un conteo de los registros que son NaN
total_crash = mask_crash.sum()          #hago una suma de esos valores "True" que me indican registros NaN, numero de registros NaN
percent_crash = 100*mask_crash.mean()   # saco la media de valores faltantes y obtengo el valor porcentual
missing_data_crash = pd.concat([total_crash, percent_crash], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
#genero un dataframe con los datos de las columnas generadas de cantidad de faltantes y porcentaje

missing_data_crash.sort_values(by='perc_missing', ascending=False, inplace=True) #ordeno por porcentaje
missing_data_crash

vemos que hay muchas columnas con una falta de datos porcentual extremadamente alta

In [ ]:
aux = pd.DataFrame(df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_1"].unique(), columns = ["type"])
aux.sort_values("type", inplace = True)
aux.reset_index(drop = True, inplace = True)
aux.head(20)

toma como eventos diferentes a: 
    
    Illness
    Illnes

    Cell Phone (hand-held)
    Cell Phone (hand-Held)

    etc.

hay que cambiarlos para que sean iguales

In [ ]:
# hago todos los reemplazos en esta serie de features (de la 1 hasta la 5)

df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_1"].replace("Illnes", "Illness", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_1"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_1"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_1"].replace("Reaction to Other Uninvolved Vehicle", "Reaction to Uninvolved Vehicle", inplace = True)

df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_2"].replace("Illnes", "Illness", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_2"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_2"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_2"].replace("Reaction to Other Uninvolved Vehicle", "Reaction to Uninvolved Vehicle", inplace = True)

df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_3"].replace("Illnes", "Illness", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_3"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_3"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_3"].replace("Reaction to Other Uninvolved Vehicle", "Reaction to Uninvolved Vehicle", inplace = True)

df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_4"].replace("Illnes", "Illness", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_4"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_4"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_4"].replace("Reaction to Other Uninvolved Vehicle", "Reaction to Uninvolved Vehicle", inplace = True)

df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_5"].replace("Illnes", "Illness", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_5"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_5"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_crash_clean["CONTRIBUTING_FACTOR_VEHICLE_5"].replace("Reaction to Other Uninvolved Vehicle", "Reaction to Uninvolved Vehicle", inplace = True)

hacer lo mismo que la feature anterior, ver errores de tipeo y arreglarlos con un replace

In [ ]:
# df_vehicle_crash_clean["VEHICLE_TYPE_CODE_1"].unique()

no se como limpiar estas features, dicen cualquier cosa

In [ ]:
aux2 = pd.DataFrame(df_vehicle_crash["VEHICLE_TYPE_CODE_1"].unique(), columns = ["type"])
aux2.sort_values("type", inplace = True)
aux2.reset_index(drop = True, inplace = True)
aux2.head(5)

In [ ]:
nullable_columns_crash = df_vehicle_crash_clean.columns[mask_crash.any()].tolist()
msno.matrix(df_vehicle_crash[nullable_columns_crash].sample(500))
plt.show()

In [ ]:
msno.heatmap(df_vehicle_crash_clean[nullable_columns_crash], figsize=(18,18))
plt.show()

## este dataset no lo convierto a .csv todavia porque tiene varias features con valores NaN

In [ ]:
# descomentar para convertir a .csv, lo guarda en el directorio donde estoy parado

# df_vehicle_crash_clean.to_csv("df_vehicle_crash_clean.csv")

##############################################################################################################################################################################################################################################

## motor-vehicle-collisions-person

https://data.world/city-of-ny/f55k-p6yu

In [ ]:
df_vehicle_person_crash = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\motor-vehicle-collisions-person.csv")

In [ ]:
df_vehicle_person_crash.columns

In [ ]:
df_vehicle_person_crash.sort_values(["CRASH_DATE", "CRASH_TIME"], inplace = True)
df_vehicle_person_crash.reset_index(drop = True, inplace = True)

df_vehicle_person_crash

In [ ]:
df_vehicle_person_crash.columns

In [ ]:
df_vehicle_person_crash["PED_ROLE"].unique()

PREGUNTAS KPI:

    porcentaje de tipos de personas afectada en la mayoria de los casos

    campaña de consientizacion sobre menores de edad lesionados por viajar en el asiento delantero

    columna COMPLAINT para una campaña de brindar informacion a hospitales, para tener en cuenta que cosas deben tomar en cuenta

PREGUNTAS KPI:

    porcentaje de tipos de personas afectada en la mayoria de los casos

FEATURES IMPORTANTES:

    'CRASH_DATE', 'CRASH_TIME'
    
FEATURES DE INFORMACION:

    PERSON_TYPE', 'PERSON_INJURY', 'PERSON_AGE', 'POSITION_IN_VEHICLE',
    'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2', 'PERSON_SEX'

AGREGAR:

    WEEK_DAY (NO ES NECESARIO PORQUE YA EXISTE EN LA OTRA TABLA)

RELACION CON OTRAS TABLAS:

    'COLLISION_ID'

In [ ]:
# fabrico el nuevo dataframe
df_vehicle_person_crash_clean = df_vehicle_person_crash[[   'CRASH_DATE', 'CRASH_TIME', 'COLLISION_ID',
                                                            'PERSON_TYPE', 'PERSON_INJURY', 'PERSON_AGE', 'POSITION_IN_VEHICLE',
                                                            'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'PED_ROLE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2', 'PERSON_SEX']]

In [ ]:
df_vehicle_person_crash_clean.head()

## HAY UN PROBLEMA GRANDE CON LAS EDADES, HAY EDADES NEGATIVAS Y EDADES COMO 9999, hay que definir un rango a partir del cual poner "Unknown"

In [ ]:
aux = pd.DataFrame(columns = ["age"], data = df_vehicle_person_crash_clean.PERSON_AGE.unique())

aux.sort_values("age")

In [ ]:
edades = df_vehicle_person_crash_clean.PERSON_AGE
edades

## aca pase los negativos a positivos, quizas se quiere directamente dropear los negativos por las dudas

In [ ]:
edades_clean = edades.apply(lambda row: row if (row >= 0) else (row * -1)) #saco los negativos
edades_clean

In [ ]:
# reasigno la columna de edades
df_vehicle_person_crash_clean["PERSON_AGE"] = edades_clean 

df_vehicle_person_crash_clean.head()

In [ ]:
df_vehicle_person_crash_clean.sort_values("PERSON_AGE")

tenemos valores de edad 0, de edad demasiado alta y de edad NaN, primero trabajo con los 0 y los altos, luego limpio los NaN

tomo edad maxima como 100 años, se puede modificar luego

In [ ]:
edades_clean = df_vehicle_person_crash_clean.PERSON_AGE #CAPTURO LAS EDADES

edades_clean = edades_clean.apply(lambda row: row if ( (row > 0) & (row <= 100) ) else "Unknown") #si se quiere tomar edad como mayor a 100, cambiar aca!

edades_clean

In [ ]:
edades_clean.fillna("Unknown", inplace = True) #saco los NaN

In [ ]:
# reasigno el valor correcto de edades pisando la columna anterior
df_vehicle_person_crash_clean["PERSON_AGE"] = edades_clean

df_vehicle_person_crash_clean.PERSON_AGE.unique()

In [ ]:
df_vehicle_person_crash_clean.PERSON_SEX.unique() #relleno los NaN con "U" (Unknown)

df_vehicle_person_crash_clean.PERSON_SEX.fillna("U", inplace = True)

In [ ]:
df_vehicle_person_crash_clean.PERSON_SEX.unique()

In [ ]:
df_vehicle_person_crash_clean.CONTRIBUTING_FACTOR_1.unique() #relleno los NaN con unspecified, el resto esta todo bien

df_vehicle_person_crash_clean.CONTRIBUTING_FACTOR_1.fillna("Unspecified", inplace = True)
df_vehicle_person_crash_clean.CONTRIBUTING_FACTOR_2.fillna("Unspecified", inplace = True)

In [ ]:
# aux = pd.DataFrame(columns = ["type"], data = df_vehicle_person_crash_clean.CONTRIBUTING_FACTOR_2.unique())

# aux.sort_values("type")

In [ ]:
df_vehicle_person_crash_clean.PED_ROLE.unique()

df_vehicle_person_crash_clean.PED_ROLE.fillna("Unknown", inplace = True) 

In [ ]:
df_vehicle_person_crash_clean.PED_ACTION.unique() #relleno los NaN con Unknown

df_vehicle_person_crash_clean.PED_ACTION.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_person_crash_clean.PED_ACTION.unique()

In [ ]:
df_vehicle_person_crash_clean.PED_LOCATION.unique() #relleno los NaN con Unknown

df_vehicle_person_crash_clean.PED_LOCATION.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_person_crash_clean.PED_LOCATION.unique() 

In [ ]:
df_vehicle_person_crash_clean.POSITION_IN_VEHICLE.unique()

viendo la columna POSITION_IN_VEHICLE, hay valores NaN y existe unknown, seteo los NaN como Unknown
tambien hay un error de tipeo en "...person&apos;s lap", lo tomo como "...person's lap"

In [ ]:
# limpieza
df_vehicle_person_crash_clean.POSITION_IN_VEHICLE.fillna("Unknown", inplace = True)
df_vehicle_person_crash_clean["POSITION_IN_VEHICLE"].replace("If one person is seated on another person&apos;s lap", "If one person is seated on another person's lap", inplace = True)

In [ ]:
df_vehicle_person_crash_clean.POSITION_IN_VEHICLE.unique()

viendo que la columna SAFETY_EQUIPMENT, mas alla de las opciones tiene NaN, Unknown y -, vamos a setear los NaN y - como Unknonwn tambien

In [ ]:
# limpieza
df_vehicle_person_crash_clean.SAFETY_EQUIPMENT.fillna("Unknown", inplace = True)
df_vehicle_person_crash_clean["SAFETY_EQUIPMENT"].replace("-", "Unknown", inplace = True)

In [ ]:
df_vehicle_person_crash_clean.SAFETY_EQUIPMENT.unique()

In [ ]:
mask_person = df_vehicle_person_crash_clean.isnull()
total_person = mask_person.sum()
percent_person = 100*mask_person.mean()
missing_data_person = pd.concat([total_person, percent_person], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
missing_data_person.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data_person

In [ ]:
# nullable_columns_person = df_vehicle_person_crash_clean.columns[mask_person.any()].tolist()
# msno.matrix(df_vehicle_person_crash_clean[nullable_columns_person].sample(500))
# plt.show()

In [ ]:
# msno.heatmap(df_vehicle_person_crash_clean[nullable_columns_person], figsize=(18,18))
# plt.show()

conversion a .csv

In [ ]:
df_vehicle_person_crash_clean.to_csv("df_vehicle_person_crash_clean.csv")

## DETERMINAR COMO ENCARAR LAS FEATURES CON MUCHOS NaN

##########################################################################################################

## motor-vehicle-collisions-vehicles

https://data.world/city-of-ny/bm4k-52h4

In [ ]:
df_vehicle_vehicle_crash = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\motor-vehicle-collisions-vehicles.csv")
df_vehicle_vehicle_crash

In [ ]:
df_vehicle_vehicle_crash.columns

## "CONTRIBUTING_FACTOR_1 y 2" ARREGLAR LA COLUMNA, TIENE VALORES IGUALES PERO CON PROBLEMAS DE TIPEO

In [ ]:
df_vehicle_vehicle_crash.CONTRIBUTING_FACTOR_1.unique()

PREGUNTAS KPI:

    probabilidad de factores que contribuyen a los choques

FEATURES IMPORTANTES:

    'UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'VEHICLE_ID', STATE_REGISTRATION, 
    TRAVEL_DIRECTION, DRIVER_SEX, DRIVER_LICENSE_STATUS, DRIVER_LICENSE_JURISDICTION, PRE_CRASH, 
    PUBLIC_PROPERTY_DAMAGE, CONTRIBUTING_FACTOR_1, CONTRIBUTING_FACTOR_1

AGREGAR:

    WEEK_DAY 

RELACION CON OTRAS TABLAS:

    'COLLISION_ID'

In [ ]:
df_vehicle_vehicle_crash_clean = df_vehicle_vehicle_crash[[     'UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'VEHICLE_ID', "STATE_REGISTRATION", 
                                                                "TRAVEL_DIRECTION", "DRIVER_SEX", "DRIVER_LICENSE_STATUS", "DRIVER_LICENSE_JURISDICTION", "PRE_CRASH", 
                                                                "PUBLIC_PROPERTY_DAMAGE", "CONTRIBUTING_FACTOR_1", "CONTRIBUTING_FACTOR_2"]]

In [ ]:
df_vehicle_vehicle_crash_clean.sort_values(["CRASH_DATE", "CRASH_TIME"], inplace = True)
df_vehicle_vehicle_crash_clean.reset_index(drop = True, inplace = True)

df_vehicle_vehicle_crash_clean.head()

## arreglo los contributing factors

In [ ]:
aux = pd.DataFrame(columns = ["type"], data = df_vehicle_vehicle_crash_clean.CONTRIBUTING_FACTOR_1.unique())
aux.sort_values("type").head(60)

## hago los reemplazos de los errores de tipeo


In [ ]:
df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_1"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_1"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_1"].replace("Illnes", "Illness", inplace = True)

df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_2"].replace("Cell Phone (hand-Held)", "Cell Phone (hand-held)", inplace = True)
df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_2"].replace("Drugs (Illegal)", "Drugs (illegal)", inplace = True)
df_vehicle_vehicle_crash_clean["CONTRIBUTING_FACTOR_2"].replace("Illnes", "Illness", inplace = True)

## ahora vamos a revisar los valores NaN en el contexto de cada columna

In [ ]:
df_vehicle_vehicle_crash_clean.head()

In [ ]:
df_vehicle_vehicle_crash_clean.DRIVER_SEX.unique()

In [ ]:
df_vehicle_vehicle_crash_clean["DRIVER_SEX"].replace("M", "Male", inplace = True)
df_vehicle_vehicle_crash_clean["DRIVER_SEX"].replace("F", "Female", inplace = True)
df_vehicle_vehicle_crash_clean["DRIVER_SEX"].replace("U", "Unknown", inplace = True)

df_vehicle_vehicle_crash_clean["DRIVER_SEX"].fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.TRAVEL_DIRECTION.unique()

reemplazo las direcciones por la palabra entera

In [ ]:
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("N", "North", inplace = True)
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("S", "South", inplace = True)
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("E", "East", inplace = True)
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("W", "West", inplace = True)
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("U", "Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].replace("-", "Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean["TRAVEL_DIRECTION"].fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.STATE_REGISTRATION.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.STATE_REGISTRATION.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.CONTRIBUTING_FACTOR_1.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.CONTRIBUTING_FACTOR_1.fillna("Unspecified", inplace = True)
df_vehicle_vehicle_crash_clean.CONTRIBUTING_FACTOR_2.fillna("Unspecified", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.PUBLIC_PROPERTY_DAMAGE.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.PUBLIC_PROPERTY_DAMAGE.fillna("Unspecified", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.PRE_CRASH.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.PRE_CRASH.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.DRIVER_LICENSE_JURISDICTION.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.DRIVER_LICENSE_JURISDICTION.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.DRIVER_LICENSE_STATUS.unique()

In [ ]:
df_vehicle_vehicle_crash_clean.DRIVER_LICENSE_STATUS.fillna("Unknown", inplace = True)

In [ ]:
df_vehicle_vehicle_crash_clean.columns

In [ ]:
mask_vehicles = df_vehicle_vehicle_crash_clean.isnull()
total_vehicles = mask_vehicles.sum()
percent_vehicles = 100*mask_vehicles.mean()
missing_data_vehicles = pd.concat([total_vehicles, percent_vehicles], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
missing_data_vehicles.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data_vehicles

In [ ]:
# nullable_columns_vehicles = df_vehicle_vehicle_crash_clean.columns[mask_vehicles.any()].tolist()
# msno.matrix(df_vehicle_vehicle_crash_clean[nullable_columns_vehicles].sample(500))
# plt.show()

In [ ]:
# msno.heatmap(df_vehicle_vehicle_crash_clean[nullable_columns_vehicles], figsize=(18,18))
# plt.show()

convertimos a .csv

In [ ]:
df_vehicle_vehicle_crash_clean.to_csv("df_vehicle_vehicle_crash_clean.csv")

## ver features con muchos NaN, determinar que hacer

##################################################################################################################

## con estos 3 dataset se puede cruzar la data para ver las zonas con mayor cantidad de siniestros viales

datasets:

        motor-vehicle-collisions-crashes        ->      df_vehicle_crash
        motor-vehicle-collisions-person         ->      df_vehicle_person_crash
        motor-vehicle-collisions-vehicles       ->      df_vehicle_vehicle_crash

## tambien se puede cruzar con la data de "traffic-volume-counts" y ver si las zonas de mayor siniestros son las zonas con mayor trafico

https://data.world/city-of-ny/btm5-ppia

el dataset https://data.cityofnewyork.us/Transportation/Automated-Traffic-Volume-Counts/7ym2-wayt es muy similar a este, solo que agrega algunas features como la hora, año, mes

es un dataset que captura el volumen de trafico en lapsos de 15 minutos, tiene 27 millones de registros

a su vez de lo puede cruzar con los dataset "bicycle-counts" y "bicycle-counts-info" para ver si las zonas de mayor trafico y choques coinciden con las zonas de mayor presencia de bicicletas

## traffic-volume-counts

In [ ]:
# capturo el dataset de trafico

df_traffic = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\traffic-volume-counts.csv")
df_traffic

In [ ]:
df_traffic.rename(columns = {"SegmentID": "Segment_ID", "Roadway Name": "Roadway_Name"}, inplace = True)
df_traffic.head()

In [ ]:
df_traffic_clean = df_traffic

## yo creo que en este dataset los valores NaN deberian reemplazarse por "0"

dado que las features con valores NaN son las de conteo, y entiendo que si un conteo resulto en NaN 
se puede interpretar como que no circularon vehiculos a esa hora...

In [ ]:
# renombro la columna para que siga los nombres del resto
df_traffic_clean.rename(columns = {"12:00-1:00 AM": "12:00-1:00AM"}, inplace = True)

In [ ]:
df_traffic_clean["12:00-1:00AM"].fillna(0, inplace = True)
df_traffic_clean["1:00-2:00AM"].fillna(0, inplace = True)
df_traffic_clean["2:00-3:00AM"].fillna(0, inplace = True)
df_traffic_clean["3:00-4:00AM"].fillna(0, inplace = True)
df_traffic_clean["4:00-5:00AM"].fillna(0, inplace = True)
df_traffic_clean["5:00-6:00AM"].fillna(0, inplace = True)
df_traffic_clean["6:00-7:00AM"].fillna(0, inplace = True)
df_traffic_clean["7:00-8:00AM"].fillna(0, inplace = True)
df_traffic_clean["8:00-9:00AM"].fillna(0, inplace = True)
df_traffic_clean["9:00-10:00AM"].fillna(0, inplace = True)
df_traffic_clean["10:00-11:00AM"].fillna(0, inplace = True)
df_traffic_clean["11:00-12:00PM"].fillna(0, inplace = True)
df_traffic_clean["12:00-1:00PM"].fillna(0, inplace = True)
df_traffic_clean["1:00-2:00PM"].fillna(0, inplace = True)
df_traffic_clean["2:00-3:00PM"].fillna(0, inplace = True)
df_traffic_clean["3:00-4:00PM"].fillna(0, inplace = True)
df_traffic_clean["4:00-5:00PM"].fillna(0, inplace = True)
df_traffic_clean["5:00-6:00PM"].fillna(0, inplace = True)
df_traffic_clean["6:00-7:00PM"].fillna(0, inplace = True)
df_traffic_clean["7:00-8:00PM"].fillna(0, inplace = True)
df_traffic_clean["8:00-9:00PM"].fillna(0, inplace = True)
df_traffic_clean["9:00-10:00PM"].fillna(0, inplace = True)
df_traffic_clean["10:00-11:00PM"].fillna(0, inplace = True)
df_traffic_clean["11:00-12:00AM"].fillna(0, inplace = True)

In [ ]:
df_traffic.columns

In [ ]:
mask_traffic = df_traffic.isnull()
total_traffic = mask_traffic.sum()
percent_traffic = 100*mask_traffic.mean()
missing_data_traffic = pd.concat([total_traffic, percent_traffic], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
missing_data_traffic.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data_traffic

## como ya no hay NaN estas celdas dan error al correrlas

In [ ]:
# nullable_columns_traffic = df_traffic.columns[mask_traffic.any()].tolist()
# msno.matrix(df_traffic[nullable_columns_traffic].sample(500))
# plt.show()

In [ ]:
# msno.heatmap(df_traffic[nullable_columns_traffic], figsize=(18,18))
# plt.show()

#################################################################################################################

capturo los dataset de las bicicletas y los uno...

https://data.world/city-of-ny/uczf-rk3c

https://data.cityofnewyork.us/Transportation/Bicycle-Counters/smn3-rzf9

In [ ]:
df_bicycle_counts = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\bicycle-counts.csv")
df_bicycle_counts.sort_values("date", inplace = True)
df_bicycle_counts.reset_index(drop = True, inplace = True)

df_bicycle_counts

este es un dataset con la expansion de la columna "id" del dataset anterior

In [ ]:
df_bicycle_counts_info = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\bicycle-counts-info.csv")

df_bicycle_counts_info.head()

hago el merge de df_bicycle_counts y df_bicycle_counts_info sobre la columna "id"

In [ ]:
df_bicycle = pd.merge(df_bicycle_counts, df_bicycle_counts_info, on = "id", how = "left")

In [ ]:
df_bicycle.head()

In [ ]:
df_bicycle.columns

In [ ]:
df_bicycle["counter"].unique()

PREGUNTAS KPI:

    

FEATURES IMPORTANTES:

    countid, id, date, counts, latitude, longitude, interval

AGREGAR:

    week_day (AGREGAR)
   

In [ ]:
df_bicycle_clean = df_bicycle[["countid", "id", "date", "counts", "latitude", "longitude", "interval"]]

In [ ]:
df_bicycle_clean.head()

In [ ]:
mask_bicycle = df_bicycle_clean.isnull()
total_bicycle = mask_bicycle.sum()
percent_bicycle = 100*mask_bicycle.mean()
missing_data_bicycle = pd.concat([total_bicycle, percent_bicycle], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
missing_data_bicycle.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data_bicycle

convertimos a .csv

In [ ]:
df_bicycle_clean.to_csv("df_bicycle_clean.csv")

##########################################################################################

## new-york-city-truck-routes

https://data.world/city-of-ny/jjja-shxy

In [ ]:
df_truck_routes = pd.read_csv(r"C:\Users\franc\OneDrive\Documentos\Henry\proyectos\PROYECTO_GRUPAL\datasets_new\new-york-city-truck-routes.csv")

df_truck_routes.head()

In [ ]:
df_truck_routes.columns

In [ ]:
df_truck_routes_clean = df_truck_routes

## NO SE QUE HACER CON ESTA FEATURE, supongo que Y sera de Yes y no se si reemplazar los NaN con N de No o Unknown

In [ ]:
df_truck_routes_clean.LtdLocal.unique()

In [ ]:
# df_truck_routes_clean.LtdLocal.fillna("N", inplace = True)
# df_truck_routes_clean.LtdLocal.fillna("Undefined", inplace = True)

## pasa lo mismo que con LtdLocal

In [ ]:
df_truck_routes_clean.ThruExwy.unique()

In [ ]:
# df_truck_routes_clean.ThruExwy.fillna("N", inplace = True)
# df_truck_routes_clean.ThruExwy.fillna("Undefined", inplace = True)

## no se que hacer con esta feature

In [ ]:
df_truck_routes_clean.Restrictio.unique()

## pasa lo mismo que con LtdLocal

In [ ]:
df_truck_routes_clean.ThruBrg.unique()

In [ ]:
# df_truck_routes_clean.ThruBrg.fillna("N", inplace = True)
# df_truck_routes_clean.ThruBrg.fillna("Undefined", inplace = True)

## pasa lo mismo que con LtdLocal

In [ ]:
df_truck_routes_clean.LocalBrg.unique()

In [ ]:
# df_truck_routes_clean.LocalBrg.fillna("N", inplace = True)
# df_truck_routes_clean.LocalBrg.fillna("Undefined", inplace = True)

## pasa lo mismo que con LtdLocal

In [ ]:
df_truck_routes_clean.ThruTunl.unique()

In [ ]:
# df_truck_routes_clean.ThruTunl.fillna("N", inplace = True)
# df_truck_routes_clean.ThruTunl.fillna("Undefined", inplace = True)

In [ ]:
df_truck_routes_clean.drop(columns = "LocalTunl", inplace = True) #la dropeo porque tiene el 100% de los valores en NaN

PREGUNTAS KPI:

    barrios con mayor densidad de trafico de camiones
    correlación entre zonas de trafico de vehiculos comunes con el trafico de camiones y bicicletas

FEATURES IMPORTANTES:

    OBJECTID_1, Street, StreetCode, RouteType, BoroName, BoroCode, 

AGREGAR:

In [ ]:
mask_bicycle = df_truck_routes_clean.isnull()
total_bicycle = mask_bicycle.sum()
percent_bicycle = 100*mask_bicycle.mean()
missing_data_bicycle = pd.concat([total_bicycle, percent_bicycle], axis=1,join='outer', keys=['count_missing', 'perc_missing'])
missing_data_bicycle.sort_values(by='perc_missing', ascending=False, inplace=True)
missing_data_bicycle

## tomar en cuenta el alcance en funcion de las fechas que aparecen en todos los datasets

## hacer un mapa de calor con las direcciones con latitud y longitud de la cantidad de choques en esa zona y tambien la cantidad de trafico

##############################################################################################################
## API DEL CLIMA

se pueden hacer hasta 1000 llamados por dia, creo que son 55 estados asi que se podria llamar a la api un par de veces al dia, o dependiendo de que info traiga se puede consultar solo 55 veces (una para cada estado) y tener esa data toda ingestada en un dataframe

1 llamado trae todo esto:

        Current weather
        Minute forecast for 1 hour
        Hourly forecast for 48 hours
        Daily forecast for 8 days
        National weather alerts
        Historical weather data for 40+ years back (since January 1, 1979)

mi API key: 03b1be2863a9f10d595a19645cbaf34d

https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}

se consulta contra ese link reemplazando los campos

In [ ]:
import requests 
import json
from pandas import json_normalize

url_climate = "https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}"
# para un testeo de consulta del clima de manhattan

lat = "40.776676"
lon = "-73.971321"

url_full = "https://api.openweathermap.org/data/2.5/weather?q=manhattan,usa&units=metric&APPID=03b1be2863a9f10d595a19645cbaf34d"

# el formato es:
# https://api.openweathermap.org/data/2.5/weather?q={ CIUDAD },{ SIGLAS PAIS }&APPID={ API KEY }

In [ ]:
response = requests.get(url_full)

In [ ]:
resp_json = response.json()

df_clima = json_normalize(resp_json) #convierto de json a dataframe

In [ ]:
df_clima

In [ ]:
df_clima["main.temp"] 
# la temperatura por defecto esta dada en kelvin, yo la pedi en sistema metrico a la API

In [ ]:
df_clima["main.feels_like"]

In [ ]:
df_clima["main.temp_min"]

In [ ]:
df_clima["main.temp_max"]

In [ ]:
df_clima.columns